In [10]:
import pandas as pd

train = pd.read_csv('UNSW_NB15_training-set.csv')
test = pd.read_csv('UNSW_NB15_testing-set.csv')

train

train.drop(['id', 'attack_cat'], axis = 1, inplace= True)
test.drop(['id', 'attack_cat'], axis = 1, inplace= True)

print(train.columns.values)
train

X = train.drop('label', axis = 1)
y = train['label']
X.proto = pd.Categorical(pd.factorize(X.proto)[0] + 1)
X.service = pd.Categorical(pd.factorize(X.service)[0] + 1)
X.state = pd.Categorical(pd.factorize(X.state)[0] + 1)
#X = pd.get_dummies(X, columns = ['proto', 'service', 'state'], drop_first = True)
X.info
display(X.dtypes)
#display(X.proto)
#X.iloc[:,:39]
num_features = X.select_dtypes(exclude=['category']).columns.values
display(num_features)
cat_features = X.select_dtypes(include=['category']).columns.values
display(cat_features)

X_t = test.drop('label', axis = 1)
y_t = test['label']
#X_t = pd.get_dummies(X_t, columns = ['proto', 'service', 'state'], drop_first = True)
X_t.proto = pd.Categorical(pd.factorize(X_t.proto)[0] + 1)
X_t.service = pd.Categorical(pd.factorize(X_t.service)[0] + 1)
X_t.state = pd.Categorical(pd.factorize(X_t.state)[0] + 1)
X_t.info
display(X_t.dtypes)
X_t.iloc[:,:39]
test_num_features = X_t.select_dtypes(exclude=['category']).columns.values
#display(test_num_features)
test_cat_features = X_t.select_dtypes(include=['category']).columns.values
display(test_cat_features)

['dur' 'proto' 'service' 'state' 'spkts' 'dpkts' 'sbytes' 'dbytes' 'rate'
 'sttl' 'dttl' 'sload' 'dload' 'sloss' 'dloss' 'sinpkt' 'dinpkt' 'sjit'
 'djit' 'swin' 'stcpb' 'dtcpb' 'dwin' 'tcprtt' 'synack' 'ackdat' 'smean'
 'dmean' 'trans_depth' 'response_body_len' 'ct_srv_src' 'ct_state_ttl'
 'ct_dst_ltm' 'ct_src_dport_ltm' 'ct_dst_sport_ltm' 'ct_dst_src_ltm'
 'is_ftp_login' 'ct_ftp_cmd' 'ct_flw_http_mthd' 'ct_src_ltm' 'ct_srv_dst'
 'is_sm_ips_ports' 'label']


dur                   float64
proto                category
service              category
state                category
spkts                   int64
dpkts                   int64
sbytes                  int64
dbytes                  int64
rate                  float64
sttl                    int64
dttl                    int64
sload                 float64
dload                 float64
sloss                   int64
dloss                   int64
sinpkt                float64
dinpkt                float64
sjit                  float64
djit                  float64
swin                    int64
stcpb                   int64
dtcpb                   int64
dwin                    int64
tcprtt                float64
synack                float64
ackdat                float64
smean                   int64
dmean                   int64
trans_depth             int64
response_body_len       int64
ct_srv_src              int64
ct_state_ttl            int64
ct_dst_ltm              int64
ct_src_dpo

array(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl',
       'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt',
       'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt',
       'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports'], dtype=object)

array(['proto', 'service', 'state'], dtype=object)

dur                   float64
proto                category
service              category
state                category
spkts                   int64
dpkts                   int64
sbytes                  int64
dbytes                  int64
rate                  float64
sttl                    int64
dttl                    int64
sload                 float64
dload                 float64
sloss                   int64
dloss                   int64
sinpkt                float64
dinpkt                float64
sjit                  float64
djit                  float64
swin                    int64
stcpb                   int64
dtcpb                   int64
dwin                    int64
tcprtt                float64
synack                float64
ackdat                float64
smean                   int64
dmean                   int64
trans_depth             int64
response_body_len       int64
ct_srv_src              int64
ct_state_ttl            int64
ct_dst_ltm              int64
ct_src_dpo

array(['proto', 'service', 'state'], dtype=object)

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier

#numeric_features = ["dur", "sbytes", "dbytes", "sttl", "dttl", "sloss", "dloss", "sload", "dload", "spkts", "dpkts", "swin", "dwin", "stcpb", "dtcpb", "trans_depth", "sjit", "djit", "tcprtt", "synack", "ackdat", "ct_state_ttl", "ct_flw_http_mthd", "ct_ftp_cmd", "ct_srv_src", "ct_srv_dst", "ct_dst_ltm", "ct_src_dport_ltm", "ct_dst_sport_ltm", "ct_dst_src_ltm"]
numeric_features = num_features
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

#categorical_features = ["proto", "state", "service"]
categorical_features = cat_features
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

lr_pl = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression(max_iter = 2000))] #incresed max_iter, default 100 wasn't enough
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

lr_pl.fit(X_train, y_train)
print("Logistic Regression model score: %.3f" % lr_pl.score(X_test, y_test))
print("Logistic Regression model test score: %.3f" % lr_pl.score(X_t, y_t))

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
svc_pl = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC(max_iter = 1000))] #increased max_iter, default 1000 wasn't enough, neither was 2000, nor 4000, nor 10000, and that's as far as I'm going
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

svc_pl.fit(X_train, y_train)
print("SVC model score: %.3f" % svc_pl.score(X_test, y_test))
print("SVC model test score: %.3f" % svc_pl.score(X_t, y_t))

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
dtc_pl = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", DTC())] 
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

dtc_pl.fit(X_train, y_train)
print("Decision Tree model score: %.3f" % dtc_pl.score(X_test, y_test))
print("Decision Tree model test score: %.3f" % dtc_pl.score(X_t, y_t))

sgd_pl = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SGDClassifier())] 
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

sgd_pl.fit(X_train, y_train)
print("Stochastic Gradient Descent model score: %.3f" % sgd_pl.score(X_test, y_test))
print("Stochastic Gradient Descent model test score: %.3f" % sgd_pl.score(X_t, y_t))

Logistic Regression model score: 0.935
Logistic Regression model test score: 0.798


C:\Users\aller\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


SVC model score: 0.935
SVC model test score: 0.557
Decision Tree model score: 0.950
Decision Tree model test score: 0.854
Stochastic Gradient Descent model score: 0.933
Stochastic Gradient Descent model test score: 0.786
